In [1]:
import traffic
import pandas as pd 
import os
from traffic.core import Traffic
from biblio_herramienta.herramienta import *
from biblio_herramienta.tratardatos import *

# 1. Carga de datos filtrados
Se cargan los datos filtrados del sector 

In [2]:
carpetaDatos = r'datos_sectores/' # carpeta que contiene los datos
nombreDatos = "datos_filtrados_def.csv"
carpetaImagenes = r"filtrado_cluster/"
vuelos = cargardatos(carpetaDatos,nombreDatos)
vuelos

,,count
icao24,callsign,
4ca60f,IBK5BV,4034
4cace5,IBK6054,3266
344495,GES381L,3216
49514c,TAP434M,3065
4cace5,IBK6055,2689
345645,VLG3981,2478
48520d,TRA845,2193
4ca25a,RYR5VC,2183
3c56f7,EWG4TP,2171


# 2. Pre-procesamiento de los datos
1. Se aplica el metedo .unwrap() para evitar que haya huecos entre 359º y 1º
2. Dentro de .clusterring se definen:
 * el algoritmo de ML
 * el metodo de preprocesamiento

In [3]:
vuelos = vuelos.assign_id().unwrap().eval(max_workers=4)
vuelos.data.head()

,Unnamed: 0,Unnamed: 0.1,alert,altitude,callsign,geoaltitude,groundspeed,hour,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate,flight_id,track_unwrapped
0,73942,6831,False,34575.0,IBE3170,35000.0,406.0,2019-02-05 07:00:00+00:00,3420cb,2019-02-05 07:29:26.872000256+00:00,42.793098,-2.902882,False,False,5325.0,2019-02-05 07:29:27+00:00,357.320625,640.0,IBE3170_097,357.320625
1,73958,6832,False,34575.0,IBE3170,35000.0,406.0,2019-02-05 07:00:00+00:00,3420cb,2019-02-05 07:29:26.872000256+00:00,42.793098,-2.902882,False,False,5325.0,2019-02-05 07:29:28+00:00,357.320625,640.0,IBE3170_097,357.320625
2,73981,6833,False,34575.0,IBE3170,35000.0,406.0,2019-02-05 07:00:00+00:00,3420cb,2019-02-05 07:29:26.872000256+00:00,42.793098,-2.902882,False,False,5325.0,2019-02-05 07:29:29+00:00,357.320625,640.0,IBE3170_097,357.320625
3,73998,6834,False,34575.0,IBE3170,35000.0,406.0,2019-02-05 07:00:00+00:00,3420cb,2019-02-05 07:29:26.872000256+00:00,42.793098,-2.902882,False,False,5325.0,2019-02-05 07:29:30+00:00,357.320625,640.0,IBE3170_097,357.320625
4,74016,6835,False,34600.0,IBE3170,35000.0,406.0,2019-02-05 07:00:00+00:00,3420cb,2019-02-05 07:29:26.872000256+00:00,42.793098,-2.902882,False,False,5325.0,2019-02-05 07:29:31+00:00,357.320625,640.0,IBE3170_097,357.320625


In [4]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from traffic.core.projection import CH1903

# la funcion permite aplicar algoritmos de ML a los datos que tenemos
vuelos_dbscan = vuelos.clustering(
    nb_samples=15, # divide en 15 puntos las trayecorias
    features=["longitude", "latitude", "track_unwrapped"], # datos cebados al algorimto de ML
    clustering=DBSCAN(eps=0.5, min_samples=10), # algoritmo de ML utilizado, con los parámetros del mismo
    transform=StandardScaler(), # lo que hace esto es escalar los datos dentro de una distribuciñ¡ón Gausiana
).fit_predict()


# 3. Representacion de los clusters y selección de aquellos que se vayan filtrar
Se representan y se eligen aquellos que no estén dentro del sector

In [ ]:
representarcluster(vuelos_dbscan,"cluster_dbscan.png","filtrado_cluster/")

Se observa que los cluster que quedan fuera son : 1, 2, 6, 8, 11, 12 y 13 

In [ ]:
vuelos_dbscan.data.info()

In [ ]:
vuelos_dbscan_f = vuelos_dbscan.data.copy()

In [ ]:
vuelos_dbscan_f.info()

# 4. Filtrado de clusters y representación de las trayectorias


In [ ]:
clusters = (-1, 0, 1, 5, 7,8, 10, 11 , 12) # se resta uno dado que se cuentan desde cero, y el -1 el total de las trayectorias
#vuelos_dbscan_F = pd.DataFrame()
for i in clusters:
    aviones_fuera_sector = vuelos_dbscan_f[vuelos_dbscan_f["cluster"]== int(i)].index
    vuelos_dbscan_f.drop(aviones_fuera_sector,inplace = True)
vuelos_dbscan_f.info()

In [ ]:
vuelos_dbscan_f = Traffic(vuelos_dbscan_f)

In [ ]:
representarSobreSector(vuelos_dbscan_f)
guardarimagen(carpetaImagenes,"sector_filtrado.png")